In [30]:
pip install shap

  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/f5/fc/e81722d6bec4fcba46e46ef895eddaeab0027ac71e78fc35ef351fac5fe4/shap-0.43.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/447.3 kB ? eta -:--:--
   -- ------------------------------------- 30.7/447.3 kB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/447.3 kB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 225.3/447.3 kB 2.3 MB/s eta 0:00:01
   ---------------------------------- ----- 389.1/447.3 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 447.3/447.3 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01

import sklearn.tree as tree

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

In [35]:
# Reading in the CSV file
dia_data = pd.read_csv("data/diabetic_data.csv")

#print(dia_X.shape)
# View the first 5 rows
print(dia_data.head())
#print(f"NumFeatures: {len(dia_data.columns)} \nFeatures: {dia_data.columns}")
#print(f"Summary: {dia_data.info}")

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [38]:
data = dia_data.to_numpy()
X, Y = data[:,:-1], data[:,-1]
#tTest20%
Xtr, Xtemp, Ytr, Ytemp = train_test_split(X, Y, test_size=0.2, random_state=seed)
#Train70%valida10%
Xva, Xtest, Yva, Ytest = train_test_split(Xtemp, Ytemp, test_size=0.25, random_state=seed)

print(Xtr.shape, Ytr.shape)
print(X.shape, Ytr.shape)

(81412, 49) (81412,)
